In [ ]:
from datetime import datetime
import sys
import time
import sqlite3
import pandas as pd
from gensim.parsing.preprocessing import strip_multiple_whitespaces, preprocess_string, remove_stopwords, strip_tags, strip_punctuation 
import gensim
import re

In [ ]:
#load
data = pd.read_parquet("data.parquet", engine="fastparquet")
original = data.copy(deep=True)

In [ ]:
def get_cat(row):
    if ("ENG - Software Engineering" in row["targeted_disciplines"]) and ("MATH - Computer Science" in row["targeted_disciplines"]):
        return True
    else:
        return False
    
def merge_duplicates(df):
    df = df.drop_duplicates(subset=['organization', 'job_title'], keep="first") 
    df = df.drop_duplicates(subset=['job_responsibilities', 'job_summary', 'required_skills'], keep="first")
    return df

def clean(x):
    newstr = strip_multiple_whitespaces(x)
    print(newstr)
    return newstr

def preprocess(x):
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, remove_stopwords, strip_multiple_whitespaces]
    x = re.sub(r"http\S+", "", x)
    x = preprocess_string(x, CUSTOM_FILTERS)
    x = " ".join(x)
    x = re.sub(r'\d+', '', x)
    return x

In [ ]:
#Clean data
data["job_responsibilities"] = data["job_responsibilities"].map(lambda x: preprocess(x))
data["job_summary"] = data["job_summary"].map(lambda x: preprocess(x))
data["required_skills"] = data["required_skills"].map(lambda x: preprocess(x))
data = merge_duplicates(data)
m = data.apply(get_cat, axis=1)
data = data[m]